<a href="https://colab.research.google.com/github/fikrifaizz/ecommerce-dashboard/blob/main/notebooks/data_profiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os

In [2]:
raw_data_path = '../data/raw/'

print(f"Path Data Raw diset ke: {os.path.abspath(raw_data_path)}")
print("List file CSV yang tersedia:")
files = [f for f in os.listdir(raw_data_path) if f.endswith('.csv')]
for f in files:
    print(f" - {f}")

Path Data Raw diset ke: /Users/fikrifaizzulfadhli/Clone Github/ecommerce-dashboard/data/raw
List file CSV yang tersedia:
 - olist_sellers_dataset.csv
 - product_category_name_translation.csv
 - olist_orders_dataset.csv
 - olist_order_items_dataset.csv
 - olist_customers_dataset.csv
 - olist_geolocation_dataset.csv
 - olist_order_payments_dataset.csv
 - olist_order_reviews_dataset.csv
 - olist_products_dataset.csv


In [3]:
def check_data_health(df, filename):
    print(f"\n{'='*60}")
    print(f"LAPORAN PROFILING: {filename}")
    print(f"{'='*60}")
    
    # 1. Dimensi Data
    rows, cols = df.shape
    print(f"Dimensi: {rows:,} Baris | {cols} Kolom")
    
    # 2. Cek Missing Values (Null)
    print("\nMissing Values (Top 10):")
    null_counts = df.isnull().sum()
    null_counts = null_counts[null_counts > 0].sort_values(ascending=False).head(10)
    
    if not null_counts.empty:
        for col, count in null_counts.items():
            percentage = (count / rows) * 100
            print(f"   - {col}: {count:,} null ({percentage:.2f}%)")
    else:
        print("   Tidak ada missing values.")

    # 3. Cek Duplikat
    dup_count = df.duplicated().sum()
    print(f"\nDuplikat Baris: {dup_count:,} baris")
    
    # 4. Cek Format Tanggal (Apakah masih Object/String?)
    # Kita cari kolom yang namanya mengandung 'date' atau 'timestamp'
    print("\nCek Tipe Data Tanggal:")
    date_cols = [col for col in df.columns if 'date' in col or 'timestamp' in col]
    if date_cols:
        for col in date_cols:
            dtype = df[col].dtype
            print(f"   - {col}: {dtype} " + ("(Harus diubah ke datetime)" if dtype == 'O' else "✅"))
    else:
        print("   Tidak ditemukan kolom tanggal berdasarkan nama.")

    # 5. DETEKSI KRITIS: Karakter 'Enter' (\n) yang merusak CSV
    # Ini penting untuk tabel Reviews
    print("\nCek Karakter Newline (Penyebab Import Error):")
    text_cols = df.select_dtypes(include=['object']).columns
    found_issue = False
    for col in text_cols:
        # Cek apakah ada karakter \n atau \r
        issue_count = df[col].astype(str).str.contains(r'[\n\r]', regex=True).sum()
        if issue_count > 0:
            print(f"   BAHAYA: Kolom '{col}' memiliki {issue_count} baris dengan 'Enter'.")
            found_issue = True
            
    if not found_issue:
        print("   Aman dari karakter newline.")

In [4]:
# List file manual (agar urutannya enak dibaca)
# Atau pakai variable 'files' dari Cell 1 jika ingin otomatis semua
target_files = [
    'olist_orders_dataset.csv',
    'olist_order_reviews_dataset.csv', # File bermasalah utama
    'olist_order_items_dataset.csv',
    'olist_products_dataset.csv',
    'olist_customers_dataset.csv',
    'olist_geolocation_dataset.csv',
    'olist_sellers_dataset.csv',
    'olist_order_payments_dataset.csv',
    'olist_order_items_dataset.csv',
]

print("MEMULAI PROSES DIAGNOSA DATA...\n")

for file_name in target_files:
    try:
        file_path = os.path.join(raw_data_path, file_name)
        
        df = pd.read_csv(file_path) 
        
        # Jalankan fungsi diagnosa
        check_data_health(df, file_name)
        
    except Exception as e:
        print(f"\nGAGAL MEMBUKA FILE: {file_name}")
        print(f"   Error Message: {e}")

MEMULAI PROSES DIAGNOSA DATA...


LAPORAN PROFILING: olist_orders_dataset.csv
Dimensi: 99,441 Baris | 8 Kolom

Missing Values (Top 10):
   - order_delivered_customer_date: 2,965 null (2.98%)
   - order_delivered_carrier_date: 1,783 null (1.79%)
   - order_approved_at: 160 null (0.16%)

Duplikat Baris: 0 baris

Cek Tipe Data Tanggal:
   - order_purchase_timestamp: object (Harus diubah ke datetime)
   - order_delivered_carrier_date: object (Harus diubah ke datetime)
   - order_delivered_customer_date: object (Harus diubah ke datetime)
   - order_estimated_delivery_date: object (Harus diubah ke datetime)

Cek Karakter Newline (Penyebab Import Error):
   Aman dari karakter newline.

LAPORAN PROFILING: olist_order_reviews_dataset.csv
Dimensi: 99,224 Baris | 7 Kolom

Missing Values (Top 10):
   - review_comment_title: 87,656 null (88.34%)
   - review_comment_message: 58,247 null (58.70%)

Duplikat Baris: 0 baris

Cek Tipe Data Tanggal:
   - review_creation_date: object (Harus diubah ke datet